# Taaltheorie en Taalverwerking · 2019 · Week 16

In [1]:
# FILL THIS IN FOR YOUR GROUP, also name your file as: tttv-w16-<group>-<name1>-<name2>.ipynb

# Group        : D
# Name - UvaID : Joshua de Roos - 11242736
# Name - UvaID : Lodewijk van Keizerswaard - 11054115
# Date         : 23-04-19

In [2]:
import nltk
from nltk import CFG
from nltk.grammar import FeatureGrammar
from nltk.parse import RecursiveDescentParser, FeatureEarleyChartParser

# Function that works for multiple types of parsers (You are free to use something else if you want.)
def check_sentence(parser, sentence):
    print("--------------------------------------------------")
    print("Checking if provided sentence matches the grammar:")
    print(sentence)
    if isinstance(sentence, str):
        sentence = sentence.split()
    tree_found = False
    results = parser.parse(sentence)
    for tree in results:
        tree_found = True
        print(tree)
    if not tree_found:
        print(sentence, "Does not match the provided grammar.")
    print("--------------------------------------------------")
    return tree_found

The main goal of this set of exercises is to implement the shift-reduce algorithm for bottom-up parsing. We will use the following grammar for most of our examples, which is a slightly modified version of the grammar we used in the exercises last week:

<table>
<tr>
    <td>Phrase structure rules</td>
    <td>Lexicon</td>
</tr>
<tr>
    <td>S $\rightarrow$ NP VP</td>
    <td>Det $\rightarrow$ *the*</td>
</tr>
<tr>
    <td>NP $\rightarrow$ Det N</td>
    <td>N $\rightarrow$ *journalist* | *detective*</td>
</tr>
<tr>
    <td>VP $\rightarrow$ V NP</td>
    <td>V $\rightarrow$ *interviews* | *photographs*</td>
</tr>
<tr>
    <td>V $\rightarrow$ V C V</td>
    <td>C $\rightarrow$ *and* | *or*</td>
</tr>
</table>


This grammar allows sentences such as *the detective interviews the journalist* as well as sentences such as *the journalist interviews and photographs the detective*.


### Question 1 (4 pts)

Encode the above grammar as a CFG named **cfg_1**, using NLTK. Then try to parse the following sentence: *The detective interviews and photographs the journalist.*


In [ ]:
# Finish the declaration of cfg_1
cfg_1 = CFG.fromstring("""
  S -> NP VP
  NP -> Det N
  VP -> V NP
  V -> V C V
  Det -> 'the'
  N -> 'journalist' | 'detective'
  V -> 'interviews' | 'photographs'
  C -> 'and' | 'or'
""")

# Use RecursiveDescentParser for this example.
cfg_1_parser = RecursiveDescentParser(cfg_1)
# The following inputs should produce the corresponding results
check_sentence(cfg_1_parser, 'the detective interviews and photographs the journalist') # RecursionError

Grammar with 11 productions (start state = S)
    S -> NP VP
    NP -> Det N
    VP -> V NP
    V -> V C V
    Det -> 'the'
    N -> 'journalist'
    N -> 'detective'
    V -> 'interviews'
    V -> 'photographs'
    C -> 'and'
    C -> 'or'
--------------------------------------------------
Checking if provided sentence matches the grammar:
the detective interviews and photographs the journalist


RecursionError: maximum recursion depth exceeded while calling a Python object

As you will see, this does not work for the *RecursiveDescentParser*.

If this does work, then try to parse the string *the detective interviews*, which should fail, and continue with the exercise. 
Trace the execution of the query. Document and explain what happens by making reference to the properties of the grammar rules and the parsing strategy employed by the CFG (_Hint_: The error traces have comments, which could help you. Try scrolling through them and try if you can see a pattern). Be thorough in your answer.

**Answers:**


The Recursive Descent parser is a top-down, depth-first parser that works from left to right. When parsing the sentence above the algorithm eventually ends up at the rule `V -> V C V`. Since this is a left-recursive rule, the algorithm will keep expanding `V` to `V C V`. This causes an infinite loop and is the reason that the RecursiveDescentParser can not parse this sentence 

Would a top-down breadth-first parsing strategy have the same problem as the *RecursiveDescentParser* parser? Explain why.

**Answers:**

No, a breadth-first parser would not have this problem. This parser would consider every possible tree at each level before expanding to the next level. So, at some point it would find the correct parsing tree for the sentence and not go in to an infinite loop.

### Question 2 (2 pts)
Encode the same grammar using the following notation. Each rule is represented by a fact of the form **rule\[Left\] = Right**, where **Left** stands for an atom representing the lefthand side of a rule, and **Right** stands for the list of terminal and nonterminal symbols on the righthand side of the rule. For the sake of keeping our sanity in Python we will not store the rules individually, but we store them in a dictionary where **Left** forms the key, and **Right** is stored in a list for each **Left**. For example, the first and the last rule of our grammar would be added as follows:


In [4]:
def add_rule(rules, left, right):
    # If the key does not already exist, initialize it with a list.
    if left not in rules:
        rules[left] = []
    rules[left].append(right)
rules = dict()
add_rule(rules, 's', ['np', 'vp'])
add_rule(rules, 'np', ['det', 'n'])
add_rule(rules, 'vp', ['v', 'np'])
add_rule(rules, 'v', ['v', 'c', 'v'])
add_rule(rules, 'det', ['the'])
add_rule(rules, 'n', ['journalist'])
add_rule(rules, 'n', ['detective'])
add_rule(rules, 'v', ['interviews'])
add_rule(rules, 'v', ['photographs'])
add_rule(rules, 'c', ['or'])
add_rule(rules, 'c', ['and'])

### Question 3 (6 pts)

Now implement the shift-reduce algorithm as a function **shift_reduce(rules, atoms_list, goal)**. The first argument holds the rules of the CFG, the second arguments represents the string of words to be parsed (a list of atoms), and the third argument represents the constituent(s) as which those words should be parsed, i.e., the parsing goal (also a list of atoms, possibly just a single atom). 

For example, this indicates that this input string of words can be parsed as a sentence: 

    shift_reduce(rules, 'the journalist interviews and interviews and interviews the detective'.split(), ['s']) # True

This indicates that this string of words cannot be parsed as the two constituents NP and VP:

    shift_reduce(rules, 'the journalist interviews'.split(), ['np', 'vp']) # False
    
Proceed as follows:
1. Start with an empty memory stack (mem_list)
2. Apply the shift operation:  remove the first word from the current sentence and shift it onto the memory stack (i.e., append it to the end of the list representing the memory component).
3. Try to reduce the memory stack as much as you can: find a rule the righthand side of which matches the last few elements on the memory stack, remove them from the memory, and instead include the lefthand side of the chosen rule at the same position in the memory component.
4. If you cannot reduce any further, shift again and so on.

When you can no longer shift and reduce, check whether the current memory component is identical with the parsing goal (goal), then you are done.

Tip: it can help to create a separate helper function for the reduce step


In [5]:
## Helper Functions (if needed)
def reduce_stack(rules, stack):
    for rule in rules:
        for expansion in rules[rule]:
            l = len(expansion)
            if expansion == stack[-l:]:
                stack = stack[:len(stack)-l]
                stack.append(rule)
                return stack
    return stack

## Actual Function
def shift_reduce(rules, atoms_list, goal):
    mem_list = []
    
    while atoms_list:
        atom = atoms_list.pop(0)
        mem_list.append(atom)
        change = True
        
        while change:
            new_mem_list = reduce_stack(rules, mem_list)
            
            if new_mem_list == mem_list:
                change = False
            else:
                mem_list = new_mem_list
            
            if mem_list == goal:
                return True
    return False

print(shift_reduce(rules, 'the journalist interviews and interviews and interviews the detective'.split(), ['s'])) #True
print(shift_reduce(rules, 'the journalist interviews'.split(), ['np', 'vp'])) # False

True
False


Document that your function **shift_reduce(rules, atoms_list, goal)** (leaving the mem\_list empty) works as intended by showing the output for three (interesting) queries (that need to be different from the queries shown above).

In [6]:
print(shift_reduce(rules, ('the detective interviews and interviews or photographs '+
                   'and photographs the journalist').split(), ['s'])) # True
print(shift_reduce(rules, 'the detective interviews and interviews the journalist'.split(), ['np', 'vp'])) #True
print(shift_reduce(rules, 'the journalist or the detective interviews the detective'.split(), ['s'])) # False

True
True
False
